In [1]:
import numpy as np
import scipy.sparse as sp

In [2]:
def readDataFromFile (fileName):
    "This functions reads data from a file and store it in two matrices"
    #Open the file
    file = open(fileName, 'r')
 
    #Now we have to read the first line and check if it's sparse or dense
    firstLine = file.readline()
    words = firstLine.split()
    word = words[1]
    if word[:-1] == 'SPARSE':
        sparse = True #The file is in sparse mode
    else:
        sparse = False #The file is in dense mode
 
 
    secondLine = file.readline()
    words = secondLine.split()
    instances = int(words[1])
    thirdLine = file.readline()
    words = thirdLine.split()
    attributes = int(words[1])
    fourthLine = file.readline()
    words = fourthLine.split()
    labels = int(words[1])
    #Now we do a loop reading all the other lines
    #Then we read the file, different way depending if sparse or dense
 
    #The loop starts in the first line of data
    #We have to store that data in two matrices
    X = np.zeros((instances, attributes), dtype=float)
    y = np.zeros((instances, labels), dtype=int)
    numberLine = 0
    for line in file.readlines():
        putToX = True
        firstIndex = 1
        numberData = 0
        numberY = 0
        for data in line.split():
            if sparse:#Sparse format, we have to split each data
                if data == '[':
                    putToX = False
 
                if putToX == True and (data != '[' and data != ']'):
                    sparseArray = data.split(':')
                    lastIndex = int(sparseArray[0])
                    for i in range(firstIndex, lastIndex - 1):
                        X[numberLine, i-1] = float(0)
                    X[numberLine, lastIndex-1] = float(sparseArray[1])
                    firstIndex = lastIndex-1
                else:
                    if (data != '[') and (data != ']'):
                        aux = float(data)
                        y[numberLine, numberY] = int(aux)
                        numberY += 1
               
            else:#Dense format
                if data == '[':
                    putToX = False
 
                if putToX == True and (data != '[' and data != ']'):
                    X[numberLine, numberData] = float(data)
                else:
                    if (data != '[') and (data != ']'):
                        #This is good for the dense format
                        aux = float(data)
                        y[numberLine, numberY] = int(aux)
                        numberY += 1
            numberData += 1
       
        numberLine += 1
    X = sp.csr_matrix(X)
    file.close()
    return X, y

In [3]:
#Sacamos los datos de train y test generados (.train y .gen)

In [4]:
#X_train.shape[0]
#y_train.shape[1]
#---------------
#X_test.shape[0]
#y_test.shape[1]

In [5]:
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.ensemble import RandomForestClassifier
import sklearn.metrics
#CHANGE DATASET NAME HERE:
dataset = 'bibtex'

In [16]:
nfolds=10
for i in range(0, nfolds):
    X_train,y_train=readDataFromFile('./datasets/'+dataset+'/'+dataset+str(i)+'.train')
    print('Reading: ./datasets/'+dataset+'/'+dataset+str(i)+'.train')
    X_test,y_test=readDataFromFile('./datasets/'+dataset+'/'+dataset+str(i)+'.test')
    print('Reading: ./datasets/'+dataset+'/'+dataset+str(i)+'.test')
    classif = BinaryRelevance(classifier=RandomForestClassifier(n_estimators=10),require_dense=[False,True])
    classif.fit(X_train,y_train)
    y_score = classif.predict(X_test)
    y_prob = classif.predict_proba(X_test)
    fold_accuracy = []
    fold_hamming = []
    fold_cover = []
    fold_rank = []
    fold_prec = []
    fold_auc = []
    folds = []
    desired_number = []
    #-----------------------------------------#
    #Medidas: sklearn.metrics...(true,predict,..)
    acc= sklearn.metrics.accuracy_score(y_test, y_score)
    fold_accuracy.append(acc)
    print ("Accuracy: %0.5f"%acc)
    #-----------------------------------------#
    hl=sklearn.metrics.hamming_loss(y_test, y_score)
    fold_hamming.append(hl)
    print("Hamming loss: %0.5f"%hl)
    #-----------------------------------------#
    #Coverage
    c=sklearn.metrics.coverage_error(y_test, y_prob.toarray(), sample_weight=None)
    fold_cover.append(c)
    print ("Coverage: %0.5f - 1"%c)
    #-----------------------------------------#
    #Ranking loss
    rl=sklearn.metrics.label_ranking_loss(y_test, y_prob.toarray(), sample_weight=None)
    fold_rank.append(rl)
    print("Ranking loss: %0.5f"%rl)
    #-----------------------------------------#
    #Mean average precision
    m=sklearn.metrics.average_precision_score(y_test, y_score.toarray(), average='macro', pos_label=1, sample_weight=None)
    fold_prec.append(m)
    print("Mean average precision: %0.5f"%m)
    #-----------------------------------------#
    #Micro-average AUC
    rmi=sklearn.metrics.roc_auc_score(y_test, y_score.toarray(), average='micro', sample_weight=None, max_fpr=None)
    fold_auc.append(rmi)
    print("ROC AUC micro: %0.5f"%rmi)
    print("-------------------")
print("Problems while trying to calculate coverage & ranking loss due to probab measure") #Happens with genbase & medical datasets

Reading: ./datasets/bibtex/bibtex0.train
Reading: ./datasets/bibtex/bibtex0.test
Accuracy: 0.10359
Hamming loss: 0.01375
Coverage: 87.10083 - 1
Ranking loss: 0.35365
Mean average precision: 0.07014
ROC AUC micro: 0.57676
-------------------
Reading: ./datasets/bibtex/bibtex1.train
Reading: ./datasets/bibtex/bibtex1.test
Accuracy: 0.10738
Hamming loss: 0.01281
Coverage: 85.93154 - 1
Ranking loss: 0.35576
Mean average precision: 0.07410
ROC AUC micro: 0.57810
-------------------
Reading: ./datasets/bibtex/bibtex2.train
Reading: ./datasets/bibtex/bibtex2.test
Accuracy: 0.09621
Hamming loss: 0.01322
Coverage: 86.95935 - 1
Ranking loss: 0.35477
Mean average precision: 0.06863
ROC AUC micro: 0.57780
-------------------
Reading: ./datasets/bibtex/bibtex3.train
Reading: ./datasets/bibtex/bibtex3.test
Accuracy: 0.09799
Hamming loss: 0.01293
Coverage: 84.58792 - 1
Ranking loss: 0.35573
Mean average precision: 0.06997
ROC AUC micro: 0.57864
-------------------
Reading: ./datasets/bibtex/bibtex4.t

In [7]:
#yscore = classif.score(X_test, y_test)

In [8]:
#y_score #prediccion de y en base al clasificador

In [9]:
print ("Avg Accuracy: ")
print sum(fold_accuracy)/len(fold_accuracy)

Avg Accuracy: 
0.10068027210884353


In [10]:
print("Hamming loss: ")
print sum(fold_hamming)/len(fold_hamming)

Hamming loss: 
0.0133401788388


In [17]:
print ("Coverage: ")
print (sum(fold_cover)/len(fold_cover))-1

Coverage: 
85.03809523809524


In [18]:
print("Ranking loss: ")
print sum(fold_rank)/len(fold_rank)

Ranking loss: 
0.35132575794229887


In [13]:
print("Mean average precision: ")
print sum(fold_prec)/len(fold_prec)

Mean average precision: 
0.07249294623248316


In [14]:
print("Micro average AUC: ")
print sum(fold_auc)/len(fold_auc)

Micro average AUC: 
0.5770460592218123


In [15]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test,y_score))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         6
           1       0.00      0.00      0.00        13
           2       0.00      0.00      0.00         7
           3       1.00      0.11      0.20         9
           4       0.00      0.00      0.00         9
           5       1.00      0.33      0.50         9
           6       0.40      0.10      0.16        20
           7       0.00      0.00      0.00         8
           8       0.00      0.00      0.00         7
           9       0.00      0.00      0.00        14
          10       0.86      0.83      0.85        30
          11       1.00      0.12      0.22         8
          12       0.00      0.00      0.00         6
          13       1.00      0.93      0.97        15
          14       0.94      0.85      0.89        52
          15       0.00      0.00      0.00         9
          16       1.00      0.09      0.17        11
          17       0.00    

/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
